**# Installation of required librairies**

In [ ]:
! pip install tqdm
! pip install numpy
! pip install sklearn
! pip install gensim
! pip install requests

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


**Download DataSet from github**

In [ ]:
!mkdir data_provider data_provider/dataset1 data_provider/dataset2 data_provider/dataset3
!wget -O /content/data_provider/train.txt https://raw.githubusercontent.com/msellamiTN/smartcities_repo/main/provider_data/train.txt
!wget -O /content/data_provider/test.txt https://raw.githubusercontent.com/msellamiTN/smartcities_repo/main/provider_data/test.txt
!wget -O /content/data_provider/valid.txt https://raw.githubusercontent.com/msellamiTN/smartcities_repo/main/provider_data/valid.txt


--2023-10-14 20:45:45--  https://raw.githubusercontent.com/msellamiTN/smartcities_repo/main/provider_data/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106732 (104K) [text/plain]
Saving to: ‘/content/data_provider/train.txt’

/content/data_provi 100%[===================>] 104.23K  --.-KB/s    in 0.03s   

2023-10-14 20:45:45 (3.61 MB/s) - ‘/content/data_provider/train.txt’ saved [106732/106732]

--2023-10-14 20:45:45--  https://raw.githubusercontent.com/msellamiTN/smartcities_repo/main/provider_data/test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent

In [ ]:
!cp data_provider/test.txt data_provider/dataset1/
!cp data_provider/test.txt data_provider/dataset2/
!cp data_provider/test.txt data_provider/dataset3/
!cp data_provider/valid.txt data_provider/dataset1/
!cp data_provider/valid.txt data_provider/dataset2/
!cp data_provider/valid.txt data_provider/dataset3/

In [ ]:
import pandas as pd
import random


# Read the dataset from the text file into a pandas DataFrame
df = pd.read_csv('/content/data_provider/train.txt', sep=' ',names=['Relations','Node1','Node2'])

# Dictionary to store the count of each node type
node_count = {
    'user': 0,
    'service': 0
}

# Dictionary to store the count of each prefix
prefix_count = {
    'SP1': 0,
    'SP2': 0,
    'SP3': 0
}

# Set to store the selected users
users = set()

# List to store the selected relations
dataset = []

# Iterate through the relations in the DataFrame
for index, row in df.iterrows():
    relation = row['Relations']
    node1 = row['Node1']
    node2 = row['Node2']

    # Skip rows with nan values
    if pd.isnull(relation) or pd.isnull(node1) or pd.isnull(node2):
        continue

    # Extract the prefix from the relation
    prefix = relation.split('_')[0]

    # Increment the count of the prefix
    prefix_count[prefix] += 1

    # If the relation contains 'U-Similar', add both nodes to the set of users
    if 'U-Similar' in relation:
        users.add(node1)
        users.add(node2)
        dataset.append((relation, node1, node2))
    # If the relation contains 'Consume' and the node1 is in the set of users, add the relation to the dataset
    elif 'Consume' in relation and node1 in users:
        dataset.append((relation, node1, node2))
    # If the relation contains 'S-Smiliar' and the node1 is in the set of services from the 'Consume' relations, add the relation to the dataset
    elif 'S-Smiliar' in relation:
        services = set()
        for _, row in df[df['Relations'].str.startswith('Consume')].iterrows():
            services.add(row['Node2'])
        if node1 in services:
            dataset.append((relation, node1, node2))

# Select a random subset of the users from the set
random.shuffle(list(users))
subset_users = set(random.sample(users, 300))

# Select a subset of the relations that contain a user from the subset of users
subset_relations = [r for r in dataset if r[1] in subset_users]

# Convert the selected relations to a DataFrame
df_subset = pd.DataFrame(subset_relations, columns=['Relations', 'Node1', 'Node2'])

# Save the DataFrame to a TSV file

df_subset.to_csv('/content/data_provider/dataset1/train.txt', sep=' ', index=False, header=False)

<ipython-input-4-a00dad115397>:61: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  subset_users = set(random.sample(users, 300))


In [ ]:
import pandas as pd
import random

def generate_subset(df, prefixes, subset_size=300, output_file='dataset.txt'):
    # Set to store the selected users
    users = set()

    # List to store the selected relations
    dataset = []

    # Iterate through the relations in the DataFrame
    for index, row in df.iterrows():
        relation = row['Relations']
        node1 = row['Node1']
        node2 = row['Node2']

        # Check if the relation prefix is in the desired prefixes
        if not any(relation.startswith(prefix) for prefix in prefixes):
            continue

        # If the relation contains 'U-Similar', add both nodes to the set of users
        if 'U-Similar' in relation:
            users.add(node1)
            users.add(node2)
            dataset.append((relation, node1, node2))
        # If the relation contains 'Consume' and the node1 is in the set of users, add the relation to the dataset
        elif 'Consume' in relation and node1 in users:
            dataset.append((relation, node1, node2))
        # If the relation contains 'S-Smiliar' and the node1 is in the set of services from the 'Consume' relations, add the relation to the dataset
        elif 'S-Smiliar' in relation:
            services = {row['Node2'] for _, row in df[df['Relations'].str.startswith('Consume')].iterrows()}
            if node1 in services:
                dataset.append((relation, node1, node2))

    # Select a random subset of the users from the set
    subset_users = set(random.sample(users, subset_size))

    # Select a subset of the relations that contain a user from the subset of users
    subset_relations = [r for r in dataset if r[1] in subset_users]

    # Convert the selected relations to a DataFrame
    df_subset = pd.DataFrame(subset_relations, columns=['Relations', 'Node1', 'Node2'])

    # Save the DataFrame to a file
    df_subset.to_csv(output_file, sep=' ', index=False, header=False)


# Main code to generate datasets
df = pd.read_csv('/content/data_provider/train.txt', sep=' ', names=['Relations', 'Node1', 'Node2'])

generate_subset(df, ['SP1'], output_file='/content/data_provider/dataset1/train.txt')
generate_subset(df, ['SP1', 'SP2'], output_file='/content/data_provider/dataset2/train.txt')
generate_subset(df, ['SP1', 'SP2', 'SP3'], output_file='/content/data_provider/dataset3/train.txt')


<ipython-input-5-9532ae924650>:36: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  subset_users = set(random.sample(users, subset_size))
<ipython-input-5-9532ae924650>:36: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  subset_users = set(random.sample(users, subset_size))
<ipython-input-5-9532ae924650>:36: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  subset_users = set(random.sample(users, subset_size))


**Gloabl Parameters of the Cross Network Training Model:**



In [ ]:
file_name = "/content/data_provider/"
epochs=epoch = 100
batch_size = 64
eval_type = "all"
dimensions = 200
edge_dim = 10
att_dim = 20
walk_length = 5
num_walks = 10
window_size = 5
negative_samples = 5
neighbor_samples = 10
patience = 5
aggregator = 'max-pooling'


**Definition of Differents walks used on the approach**

In [ ]:
import itertools
import random
import networkx as nx
import numpy as np
from joblib import Parallel, delayed

def preprocess_nxgraph(graph):
    node2idx = {}
    idx2node = []
    node_size = 0
    for node in graph.nodes():
        node2idx[node] = node_size
        idx2node.append(node)
        node_size += 1
    return idx2node, node2idx

def partition_dict(vertices, workers):
    batch_size = (len(vertices) - 1) // workers + 1
    part_list = []
    part = []
    count = 0
    for v1, nbs in vertices.items():
        part.append((v1, nbs))
        count += 1
        if count % batch_size == 0:
            part_list.append(part)
            part = []
    if len(part) > 0:
        part_list.append(part)
    return part_list


def partition_list(vertices, workers):
    batch_size = (len(vertices) - 1) // workers + 1
    part_list = []
    part = []
    count = 0
    for v1, nbs in enumerate(vertices):
        part.append((v1, nbs))
        count += 1
        if count % batch_size == 0:
            part_list.append(part)
            part = []
    if len(part) > 0:
        part_list.append(part)
    return part_list

def partition_num(num, workers):
    if num % workers == 0:
        return [num//workers]*workers
    else:
        return [num//workers]*workers + [num % workers]

class RWGraph():
    def __init__(self, nx_G, node_type=None):
        self.G = nx_G
        self.node_type = node_type
    def deepwalk_walk(self, walk_length, start_node):

        walk = [start_node]

        while len(walk) < walk_length:
            cur = walk[-1]
            cur_nbrs = list(self.G.neighbors(cur))
            if len(cur_nbrs) > 0:
                walk.append(random.choice(cur_nbrs))
            else:
                break
        return walk

    def node2vec_walk(self, walk_length, start_node):

        G = self.G
        alias_nodes = self.alias_nodes
        alias_edges = self.alias_edges

        walk = [start_node]

        while len(walk) < walk_length:
            cur = walk[-1]
            cur_nbrs = list(G.neighbors(cur))
            if len(cur_nbrs) > 0:
                if len(walk) == 1:
                    walk.append(
                        cur_nbrs[alias_sample(alias_nodes[cur][0], alias_nodes[cur][1])])
                else:
                    prev = walk[-2]
                    edge = (prev, cur)
                    next_node = cur_nbrs[alias_sample(alias_edges[edge][0],
                                                      alias_edges[edge][1])]
                    walk.append(next_node)
            else:
                break

        return walk

    def simulate_walks(self, num_walks, walk_length, workers, verbose=0):

        G = self.G

        nodes = list(G.nodes())

        results = Parallel(n_jobs=workers, verbose=verbose, )(
            delayed(self._simulate_walks)(nodes, num, walk_length) for num in
            partition_num(num_walks, workers))

        walks = list(itertools.chain(*results))

        return walks

    def _simulate_walks(self, nodes, num_walks, walk_length,):
        walks = []
        for _ in range(num_walks):
            random.shuffle(nodes)
            for v in nodes:
                walks.append(self.deepwalk_walk(walk_length=walk_length, start_node=v))

        return walks

**Definition of required methods and tools used on the approach**

In [ ]:
import argparse
from collections import defaultdict
import joblib
import networkx as nx
import numpy as np
from gensim.models.keyedvectors import Vocab
from six import iteritems
from sklearn.metrics import (auc, f1_score, precision_recall_curve,
                             roc_auc_score)




def get_G_from_edges(edges):
    edge_dict = dict()
    for edge in edges:
        edge_key = str(edge[0]) + '_' + str(edge[1])
        if edge_key not in edge_dict:
            edge_dict[edge_key] = 1
        else:
            edge_dict[edge_key] += 1
    tmp_G = nx.Graph()
    for edge_key in edge_dict:
        weight = edge_dict[edge_key]
        x = edge_key.split('_')[0]
        y = edge_key.split('_')[1]
        tmp_G.add_edge(x, y)
        tmp_G[x][y]['weight'] = weight
    return tmp_G

def load_training_data(f_name):
    print('We are loading data from:', f_name)
    edge_data_by_type = dict()
    all_nodes = list()
    with open(f_name, 'r') as f:
        for line in f:
            words = line[:-1].split(' ')
            if words[0] not in edge_data_by_type:
                edge_data_by_type[words[0]] = list()
            x, y = words[1], words[2]
            edge_data_by_type[words[0]].append((x, y))
            all_nodes.append(x)
            all_nodes.append(y)
    all_nodes = list(set(all_nodes))

    print('Total training nodes: ' + str(len(all_nodes)))
    return edge_data_by_type


def load_testing_data(f_name):
    print('We are loading data from:', f_name)
    true_edge_data_by_type = dict()
    false_edge_data_by_type = dict()
    all_edges = list()
    all_nodes = list()
    with open(f_name, 'r') as f:
        #i=0
        for line in f:
            #i=i+1
            #print("**line:", line, "  i=", i)
            words = line[:-1].split(' ')
            x, y = words[1], words[2]
            if int(words[3]) == 1:
                if words[0] not in true_edge_data_by_type:
                    true_edge_data_by_type[words[0]] = list()
                true_edge_data_by_type[words[0]].append((x, y))
            else:
                if words[0] not in false_edge_data_by_type:
                    false_edge_data_by_type[words[0]] = list()
                false_edge_data_by_type[words[0]].append((x, y))
            all_nodes.append(x)
            all_nodes.append(y)
    all_nodes = list(set(all_nodes))
    return true_edge_data_by_type, false_edge_data_by_type

def generate_walks(network_data, num_walks, walk_length, file_name):
    workers = joblib.cpu_count()
    print('number of cpu: '+str(workers))
    all_walks = []
    for layer_id in network_data:
        tmp_data = network_data[layer_id]
        # start to do the random walk on a layer
        layer_walker = RWGraph(get_G_from_edges(tmp_data))
        layer_walks = layer_walker.simulate_walks(num_walks, walk_length, workers)

        all_walks.append(layer_walks)

    print('Finish generating the walks')

    return all_walks

def generate_pairs(all_walks, vocab, window_size):
    pairs = []
    skip_window = window_size // 2
    for layer_id, walks in enumerate(all_walks):
        for walk in walks:
            for i in range(len(walk)):
                for j in range(1, skip_window + 1):
                    if i - j >= 0:
                        pairs.append((vocab[walk[i]].index, vocab[walk[i - j]].index, layer_id))
                    if i + j < len(walk):
                        pairs.append((vocab[walk[i]].index, vocab[walk[i + j]].index, layer_id))
    return pairs

def generate_vocab(all_walks):
    index2word = []
    raw_vocab = defaultdict(int)

    for walks in all_walks:
        for walk in walks:
            for word in walk:
                raw_vocab[word] += 1

    vocab = {}
    for word, v in iteritems(raw_vocab):
        vocab[word] = Vocab(count=v, index=len(index2word))
        index2word.append(word)

    index2word.sort(key=lambda word: vocab[word].count, reverse=True)
    for i, word in enumerate(index2word):
        vocab[word].index = i

    return vocab, index2word

def get_score(local_model, node1, node2):
    try:
        vector1 = local_model[node1]
        vector2 = local_model[node2]
        return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    except Exception as e:
        pass


def evaluate(model, true_edges, false_edges):
    true_list = list()
    prediction_list = list()
    true_num = 0
    for edge in true_edges:
        tmp_score = get_score(model, str(edge[0]), str(edge[1]))
        if tmp_score is not None:
            true_list.append(1)
            prediction_list.append(tmp_score)
            true_num += 1
    for edge in false_edges:
        tmp_score = get_score(model, str(edge[0]), str(edge[1]))
        if tmp_score is not None:
            true_list.append(0)
            prediction_list.append(tmp_score)
    sorted_pred = prediction_list[:]
    sorted_pred.sort()
    threshold = sorted_pred[-true_num]
    y_pred = np.zeros(len(prediction_list), dtype=np.int32)
    for i in range(len(prediction_list)):
        if prediction_list[i] >= threshold:
            y_pred[i] = 1
    y_true = np.array(true_list)
    y_scores = np.array(prediction_list)
    ps, rs, _ = precision_recall_curve(y_true, y_scores)
    return roc_auc_score(y_true, y_scores), f1_score(y_true, y_pred), auc(rs, ps)

In [ ]:
import math
import os
import pickle
import sys
import time
from numpy.random import seed
import numpy as np
# to deal with numpy randomness
# seed(1234)
import tensorflow.compat.v1 as tf
# added to deal with randomness
tf.set_random_seed(1234)

import tqdm
from numpy import random

#from utils import *
def get_cosine_similarity(embeddings1, embeddings2, node):
    try:
        vector1 = embeddings1[node]
        vector2 = embeddings2[node]
        return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    except:
        return 2+random.random()

def get_batches(pairs, neighbors, batch_size):
    n_batches = (len(pairs) + (batch_size - 1)) // batch_size

    for idx in range(n_batches):
        x, y, t, neigh = [], [], [], []
        for i in range(batch_size):
            index = idx * batch_size + i
            if index >= len(pairs):
                break
            x.append(pairs[index][0])
            y.append(pairs[index][1])
            t.append(pairs[index][2])
            neigh.append(neighbors[pairs[index][0]])
        yield (np.array(x).astype(np.int32), np.array(y).reshape(-1, 1).astype(np.int32), np.array(t).astype(np.int32), np.array(neigh).astype(np.int32))

def train_model(network_data, log_name, store_file, all_walks):

    # Record the start time
    start_time = time.time()
    vocab, index2word = generate_vocab(all_walks)
    train_pairs = generate_pairs(all_walks, vocab, window_size)

    edge_types = list(network_data.keys())
    print('edge types: '+str(list(network_data.keys())))
    num_nodes = len(index2word)
    edge_type_count = len(edge_types)
    print('edge types '+ str(edge_types))
    embedding_size = dimensions # Dimension of the embedding vector.
    embedding_u_size =edge_dim
    u_num = edge_type_count
    num_sampled = negative_samples # Number of negative examples to sample.
    dim_a = att_dim
    att_head = 1
    neighbors = [[[] for __ in range(edge_type_count)] for _ in range(num_nodes)]
    for r in range(edge_type_count):
        g = network_data[edge_types[r]]
        for (x, y) in g:
            ix = vocab[x].index
            iy = vocab[y].index
            neighbors[ix][r].append(iy)
            neighbors[iy][r].append(ix)
        for i in range(num_nodes):
            if len(neighbors[i][r]) == 0:
                neighbors[i][r] = [i] * neighbor_samples
            elif len(neighbors[i][r]) < neighbor_samples:
                neighbors[i][r].extend(list(np.random.choice(neighbors[i][r], size=neighbor_samples-len(neighbors[i][r]))))
            elif len(neighbors[i][r]) > neighbor_samples:
                neighbors[i][r] = list(np.random.choice(neighbors[i][r], size=neighbor_samples))

    graph = tf.Graph()


    with graph.as_default():
        global_step = tf.Variable(0, name='global_step', trainable=False)


        # Parameters to learn
        node_embeddings = tf.Variable(tf.random.uniform([num_nodes, embedding_size], -1.0, 1.0))
        print(node_embeddings.get_shape().as_list())
        node_type_embeddings = tf.Variable(tf.random.uniform([num_nodes, u_num, embedding_u_size], -1.0, 1.0))
        print(node_type_embeddings.get_shape().as_list())
        trans_weights = tf.Variable(tf.truncated_normal([edge_type_count, embedding_u_size, embedding_size // att_head], stddev=1.0 / math.sqrt(embedding_size)))
        trans_weights_s1 = tf.Variable(tf.truncated_normal([edge_type_count, embedding_u_size, dim_a], stddev=1.0 / math.sqrt(embedding_size)))
        trans_weights_s2 = tf.Variable(tf.truncated_normal([edge_type_count, dim_a, att_head], stddev=1.0 / math.sqrt(embedding_size)))
        nce_weights = tf.Variable(tf.truncated_normal([num_nodes, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([num_nodes]))

        # Input data
        train_inputs = tf.placeholder(tf.int32, shape=[None])
        train_labels = tf.placeholder(tf.int32, shape=[None, 1])
        train_types = tf.placeholder(tf.int32, shape=[None])
        node_neigh = tf.placeholder(tf.int32, shape=[None, edge_type_count, neighbor_samples])

        # Look up embeddings for nodes
        node_embed = tf.nn.embedding_lookup(node_embeddings, train_inputs)
        # to get neighbors embeddings for all nodes in all types of edges: embedding_lookup(params, ids)
        node_embed_neighbors = tf.nn.embedding_lookup(node_type_embeddings, node_neigh)

        if aggregator == 'max-pooling':
            node_embed_tmp = tf.concat([tf.reshape(tf.slice(node_embed_neighbors, [0, i, 0, i, 0], [-1, 1, -1, 1, -1]), [1, -1, neighbor_samples, embedding_u_size]) for i in range(edge_type_count)], axis=0)
            node_type_embed = tf.transpose(tf.squeeze(tf.nn.max_pool(node_embed_tmp, ksize=[1, 1, neighbor_samples, 1], strides=[1, 1, neighbor_samples, 1], padding='VALID', data_format='NHWC'), [2]), perm=[1,0,2])
            trans_w = tf.nn.embedding_lookup(trans_weights, train_types)
            trans_w_s1 = tf.nn.embedding_lookup(trans_weights_s1, train_types)
            trans_w_s2 = tf.nn.embedding_lookup(trans_weights_s2, train_types)
            attention = tf.reshape(tf.nn.softmax(tf.reshape(tf.matmul(tf.tanh(tf.matmul(node_type_embed, trans_w_s1)), trans_w_s2), [-1, u_num])), [-1, att_head, u_num])
            node_type_embed = tf.matmul(attention, node_type_embed)
            node_embed = node_embed + tf.reshape(tf.matmul(node_type_embed, trans_w), [-1, embedding_size])

        elif aggregator == 'mean':
            node_embed_tmp = tf.concat([tf.reshape(tf.slice(node_embed_neighbors, [0, i, 0, i, 0], [-1, 1, -1, 1, -1]), [1, -1, neighbor_samples, embedding_u_size]) for i in range(edge_type_count)], axis=0)
            node_type_embed = tf.transpose(tf.reduce_mean(node_embed_tmp, axis=2), perm=[1,0,2])
            trans_w = tf.nn.embedding_lookup(trans_weights, train_types)
            trans_w_s1 = tf.nn.embedding_lookup(trans_weights_s1, train_types)
            trans_w_s2 = tf.nn.embedding_lookup(trans_weights_s2, train_types)
            attention = tf.reshape(tf.nn.softmax(tf.reshape(tf.matmul(tf.tanh(tf.matmul(node_type_embed, trans_w_s1)), trans_w_s2), [-1, u_num])), [-1, att_head, u_num])
            node_type_embed = tf.matmul(attention, node_type_embed)
            node_embed = node_embed + tf.reshape(tf.matmul(node_type_embed, trans_w), [-1, embedding_size])

        elif aggregator == 'LSTM':
            node_embed_tmp = tf.concat([tf.reshape(tf.slice(node_embed_neighbors, [0, i, 0, i, 0], [-1, 1, -1, 1, -1]), [1, -1, neighbor_samples, embedding_u_size]) for i in range(edge_type_count)], axis=0)
            shape = [node_embed_tmp.shape[k] for k in range(4)]
            Y=tf.reshape(node_embed_tmp, [-1, shape[2], shape[3]])
            lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(embedding_u_size)
            node_type_emb, _ = tf.nn.dynamic_rnn(lstm_cell, Y, dtype=tf.float32)
            node_type_emb = tf.transpose(node_type_emb, [1, 0, 2])
            last = tf.gather(node_type_emb, int(node_type_emb.get_shape()[0]) - 1)
            last=tf.reshape(last, [-1, edge_type_count, embedding_u_size])
            trans_w = tf.nn.embedding_lookup(trans_weights, train_types)
            trans_w_s1 = tf.nn.embedding_lookup(trans_weights_s1, train_types)
            trans_w_s2 = tf.nn.embedding_lookup(trans_weights_s2, train_types)

            attention = tf.reshape(tf.nn.softmax(tf.reshape(tf.matmul(tf.tanh(tf.matmul(last, trans_w_s1)), trans_w_s2), [-1, u_num])), [-1, att_head, u_num])
            last = tf.matmul(attention, last)
            node_embed = node_embed + tf.reshape(tf.matmul(last, trans_w), [-1, embedding_size])


        last_node_embed = tf.nn.l2_normalize(node_embed, axis=1)

        loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                labels=train_labels,
                inputs=last_node_embed,
                num_sampled=num_sampled,
                num_classes=num_nodes))
        plot_loss = tf.summary.scalar("loss", loss)

        # Optimizer.
        optimizer = tf.train.AdamOptimizer().minimize(loss, global_step=global_step)

        # Add ops to save and restore all the variables.
        # saver = tf.train.Saver(max_to_keep=20)

        merged = tf.summary.merge_all(key=tf.GraphKeys.SUMMARIES)

        # Initializing the variables
        init = tf.global_variables_initializer()
      # Create an empty DataFrame to store results   average_auc average_f1 average_pr
    results_df = pd.DataFrame(columns=['Epoch', 'Iteration', 'Average Loss', 'Loss Value','Average AUC', 'Average F1', 'Loss PR'])


    # Launch the graph
    print("Optimizing")

    with tf.Session(graph=graph) as sess:
        writer = tf.summary.FileWriter("./runs/" + log_name, sess.graph) # tensorboard --logdir=./runs
        sess.run(init)

        print('Training')
        g_iter = 0
        best_score = 0
        patience = 0
        for epoch in range(epochs):
            random.shuffle(train_pairs)
            batches = get_batches(train_pairs, neighbors, batch_size)

            data_iter = tqdm.tqdm(batches,
                                desc="epoch %d" % (epoch),
                                total=(len(train_pairs) + (batch_size - 1)) // batch_size,
                                bar_format="{l_bar}{r_bar}")
            avg_loss = 0.0

            for i, data in enumerate(data_iter):
                feed_dict = {train_inputs: data[0], train_labels: data[1], train_types: data[2], node_neigh: data[3]}
                _, loss_value, summary_str = sess.run([optimizer, loss, merged], feed_dict)
                writer.add_summary(summary_str, g_iter)

                g_iter += 1

                avg_loss += loss_value

                if i % 5000 == 0:
                    post_fix = {
                        "epoch": epoch,
                        "iter": i,
                        "avg_loss": avg_loss / (i + 1),
                        "loss": loss_value
                    }
                    data_iter.write(str(post_fix))

            final_model = dict(zip(edge_types, [dict() for _ in range(edge_type_count)]))
            for i in range(edge_type_count):
                for j in range(num_nodes):
                    final_model[edge_types[i]][index2word[j]] = np.array(sess.run(last_node_embed, {train_inputs: [j], train_types: [i], node_neigh: [neighbors[j]]})[0])

            valid_aucs, valid_f1s, valid_prs = [], [], []
            test_aucs, test_f1s, test_prs = [], [], []
            for i in range(edge_type_count):
                if eval_type == 'all' or edge_types[i] in eval_type.split(','):
                    tmp_auc, tmp_f1, tmp_pr = evaluate(final_model[edge_types[i]], valid_true_data_by_edge[edge_types[i]], valid_false_data_by_edge[edge_types[i]])
                    valid_aucs.append(tmp_auc)
                    valid_f1s.append(tmp_f1)
                    valid_prs.append(tmp_pr)

                    tmp_auc, tmp_f1, tmp_pr = evaluate(final_model[edge_types[i]], testing_true_data_by_edge[edge_types[i]], testing_false_data_by_edge[edge_types[i]])
                    test_aucs.append(tmp_auc)
                    test_f1s.append(tmp_f1)
                    test_prs.append(tmp_pr)
            print('valid auc:', np.mean(valid_aucs))
            print('valid pr:', np.mean(valid_prs))
            print('valid f1:', np.mean(valid_f1s))

            average_auc = np.mean(test_aucs)
            average_f1 = np.mean(test_f1s)
            average_pr = np.mean(test_prs)
              # Append the result to the DataFrame
            results = {
                                  "epoch": epoch,
                                  "iter": i,
                                  "avg_loss": avg_loss / (i + 1),
                                  "loss": loss_value,
                                  "average_auc":average_auc,
                                  "average_f1" :average_f1,
                                  "average_pr" :average_f1,
                                  "start_time":start_time,
                                  "end_time": time.time()
                              }
            results_df = results_df.append(results, ignore_index=True)
            cur_score = np.mean(valid_aucs)
            if cur_score > best_score:
                best_score = cur_score
                patience = 0
            else:
                patience += 1
                if patience > patience:
                    print('Early Stopping')
                    break
    # Record the end time
    end_time = time.time()

    # Compute the total training time
    training_duration = end_time - start_time
    print(f"Training took {training_duration:.2f} seconds")
    return final_model, average_auc, average_f1, average_pr,results_df,training_duration





# Cross Network Enbedding

In [ ]:
if __name__ == "__main__":


    log_name = file_name.split('/')[-1] + f'_evaltype_{eval_type}_b_{batch_size}_e_{epoch}'

    training_data_by_type = load_training_data(file_name + '/train.txt')
    valid_true_data_by_edge, valid_false_data_by_edge = load_testing_data(file_name + '/valid.txt')
    testing_true_data_by_edge, testing_false_data_by_edge = load_testing_data(file_name + '/test.txt')
    all_walks = generate_walks(training_data_by_type, num_walks, walk_length, file_name)
    final_model, average_auc, average_f1, average_pr,results_df,training_duration = train_model(training_data_by_type, log_name + '_' + time.strftime('%Y-%m-%d %H-%M-%S',time.localtime(time.time())), 'embeddings1.txt', all_walks)

    print('Overall ROC-AUC:', average_auc)
    print('Overall PR-AUC', average_pr)
    print('Overall F1:', average_f1)
    print('Training Duration :', training_duration)
     # Save the results DataFrame to a CSV file
    results_df.to_csv('results.csv', index=False)
    #Use dumps to convert the object to a serialized string
    pickle.dump(final_model, open('final_model.emb', 'wb'))
    #Use loads to de-serialize an object
    #final_model = pickle.load(open('final_model.emb', 'rb'))

We are loading data from: /content/data_provider//train.txt
Total training nodes: 1620
We are loading data from: /content/data_provider//valid.txt
We are loading data from: /content/data_provider//test.txt
number of cpu: 2
Finish generating the walks
edge types: ['SP1_S-Smiliar', 'SP1_Consume', 'SP2_U-Similar', 'SP2_S-Smiliar', 'SP2_Consume', 'SP1_U-Similar', 'SP3_S-Smiliar', 'SP3_U-Similar', 'SP3_Consume']
edge types ['SP1_S-Smiliar', 'SP1_Consume', 'SP2_U-Similar', 'SP2_S-Smiliar', 'SP2_Consume', 'SP1_U-Similar', 'SP3_S-Smiliar', 'SP3_U-Similar', 'SP3_Consume']
[1620, 200]
[1620, 9, 10]
Optimizing
Training


epoch 0:   0%|| 5/6644 [00:00<13:05,  8.45it/s]  

{'epoch': 0, 'iter': 0, 'avg_loss': 18.928958892822266, 'loss': 18.928959}


epoch 0:  75%|| 5006/6644 [02:16<00:48, 33.82it/s]

{'epoch': 0, 'iter': 5000, 'avg_loss': 13.257196434126737, 'loss': 6.43342}


...

epoch 54: 100%|| 6644/6644 [02:18<00:00, 47.94it/s]
<ipython-input-9-1f5b8b619273>:244: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(results, ignore_index=True)


valid auc: 0.615544991825364
valid pr: 0.6918559530725885
valid f1: 0.6205778327524337


epoch 55:   0%|| 0/6644 [00:00<?, ?it/s]

{'epoch': 55, 'iter': 0, 'avg_loss': 0.11079271137714386, 'loss': 0.11079271}


epoch 55:  11%|| 715/6644 [00:15<01:53, 52.30it/s]

In [ ]:
print('Overall ROC-AUC:', average_auc)
print('Overall PR-AUC', average_pr)
print('Overall F1:', average_f1)
print('Training Duration :', training_duration)
  # Save the results DataFrame to a CSV file
results_df.to_csv('results.csv', index=False)
#Use dumps to convert the object to a serialized string
pickle.dump(final_model, open('final_model.emb', 'wb'))
#Use loads to de-serialize an object
#final_model = pickle.load(open('final_model.emb', 'rb')

In [ ]:
import pandas as pd

# Read the .tsv file
df = pd.read_csv('results.csv', delimiter=',')
df.dropna()
df=df[['epoch', 'iter', 'avg_loss', 'loss','average_auc', 'average_f1', 'average_pr', 'start_time', 'end_time']]
df

In [ ]:
df.columns = [col.strip() for col in df.columns]


In [ ]:
#['epoch', 'iter', 'avg_loss', 'loss','average_auc', 'average_f1', 'average_pr', 'start_time', 'end_time']
# Create LaTeX tikzpicture plot from DataFrame
def generate_latex_plot(df, y_column, ylabel):
    coordinates = "\n".join(f"({row['epoch']}, {row[y_column]})" for _, row in df.iterrows())
    return f"""
\\begin{{tikzpicture}}
\\begin{{axis}}[
    title={{{ylabel} vs. Epoch}},
    xlabel={{Epoch}},
    ylabel={{{ylabel}}},
    grid=major,
    legend entries={{{ylabel}}},
]
\\addplot coordinates {{
{coordinates}
}};
\\end{{axis}}
\\end{{tikzpicture}}
"""

# Generating the plots
auc_plot = generate_latex_plot(df, 'average_auc', 'average_auc')
pr_plot = generate_latex_plot(df, 'average_pr', 'average_pr')
f1_plot = generate_latex_plot(df, 'average_f1', 'average_f1')

latex_script = f"""
\\documentclass{{standalone}}
\\usepackage{{pgfplots}}
\\pgfplotsset{{compat=1.15}}

\\begin{{document}}
{auc_plot}
{pr_plot}
{f1_plot}
\\end{{document}}
"""

# Save to a .tex file
with open('plots.tex', 'w') as file:
    file.write(latex_script)

print("LaTeX script saved to plots.tex")


In [ ]:
# save the model to disk
filename = 'final_model.emb'
pickle.dump(final_model, open(filename, 'wb'))

In [ ]:
final_model.keys()

In [ ]:
#(final_model.values())


In [ ]:
import numpy as np
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def LocateAnchor(active_user_id, network):
    """Locate the embedding of the active user in the given network."""
    return network.get(active_user_id, None)

def Neighbors(anchor_embedding, user_network):
    """Retrieve embeddings of users who are similar to the anchor."""
    similar_users_embeddings = [emb for user, emb in user_network.items() if sigmoid(np.dot(anchor_embedding, emb)) > 0.5]
    return similar_users_embeddings

def services(user_embedding, service_network):
    """Return services consumed by users who have similar embeddings."""
    return [service for service, service_emb in service_network.items() if sigmoid(np.dot(user_embedding, service_emb)) > 0.5]

def Rating(user_embedding, service_embedding, alpha=0.5):
    if user_embedding is None or service_embedding is None:
        return 0
    return alpha * sigmoid(np.dot(user_embedding, service_embedding))


In [ ]:
def cross_network_service_recommendation(embeddings, active_user_id, top_k=5):
    S_top_k = {}

    for network_type, network in embeddings.items():
        # Work with user similarity networks
        if 'U-Similar' in network_type:
            anchor_embedding = LocateAnchor(active_user_id, network)

            if anchor_embedding is not None:
                similar_users_embeddings = Neighbors(anchor_embedding, network)

                service_network_type = network_type.replace('U-Similar', 'Consume')
                service_network = embeddings[service_network_type]

                for user_embedding in similar_users_embeddings:
                    user_services = services(user_embedding, service_network)

                    for service in user_services:
                        if service not in S_top_k:
                            S_top_k[service] = Rating(anchor_embedding, service_network[service])

    # Sort services based on ratings and return top-k services
    sorted_services = sorted(S_top_k, key=S_top_k.get, reverse=True)

    return sorted_services[:top_k]


In [ ]:
import pickle

with open("/content/final_model.emb", "rb") as file:
    final_model = pickle.load(file)


In [ ]:
# Suppose final_model is your embeddings dictionary
top_services = cross_network_service_recommendation(final_model, '470500', top_k=5)
print(top_services)


In [ ]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def LocateAnchor(active_user, user_network):
    return user_network.get(active_user, None)

def Neighbors(anchor_embedding, user_similarity_network):
    similar_users = [user for user, embedding in user_similarity_network.items()
                     if sigmoid(np.dot(anchor_embedding, embedding)) > 0.5]
    return similar_users

def Services(user, service_network):
    # Assuming the service network stores services consumed by each user in a list

    return service_network.get(user, [])

def Rating(user_embedding, service_embedding, alpha=0.5):
    """Compute the rating for a user-service pair."""
    # Given the context, we're treating the same embedding as both v and x (or y for services).
    latent_factor_product = np.dot(user_embedding, service_embedding)
    sigmoid_product = sigmoid(np.dot(user_embedding, service_embedding))
    return latent_factor_product + alpha * sigmoid_product

def cross_network_service_recommendation(embeddings, active_user, top_k=10,dataset_nb=4):
    ratings = {}

    # Error checks
    if not isinstance(embeddings, dict):
        raise ValueError("The embeddings parameter should be a dictionary.")
    for i in range(1, dataset_nb):
        if f'SP{i}_U-Similar' not in embeddings or f'SP{i}_Consume' not in embeddings:
            raise KeyError(f"Embeddings dictionary is missing keys for SP{i}. Ensure it has both SP{i}_U-Similar and SP{i}_Consume.")

    # Main recommendation logic
    for i in range(1, dataset_nb):
        user_similarity_network = embeddings[f'SP{i}_U-Similar']
        service_network = embeddings[f'SP{i}_Consume']

        active_user_embedding = LocateAnchor(active_user, user_similarity_network)
        if active_user_embedding is None:
            continue

        similar_users = Neighbors(active_user_embedding, user_similarity_network)
        for user in similar_users:
            consumed_services = Services(user, service_network)
            #print(consumed_services)
            for service, service_embedding in service_network.items():

                if service not in consumed_services: # Avoid recommending already recommanded services
                    ratings[service] = Rating(active_user_embedding, service_embedding)

    # Sort services by their scores to get top recommendations
    sorted_ratings = sorted(ratings.items(), key=lambda x: x[1], reverse=True)
    return sorted_ratings[:top_k]

# Example usage:
# final_model = Your embeddings dictionary
top_k_services = cross_network_service_recommendation(final_model, '470500', top_k=4,dataset_nb=3)

print(top_k_services)


In [ ]:
import matplotlib.pyplot as plt

def get_average_score(top_k_services):
    """
    Compute the average rating from the top k services.
    """
    total_score = sum([score for _, score in top_k_services])
    return total_score / len(top_k_services)

# Varying k values
k_values = list(range(2, 11, 2))

# Storing average scores
average_scores = []

for k in k_values:
    top_k_services = cross_network_service_recommendation(final_model, '470500', top_k=k,dataset_nb=3)
    avg_score = get_average_score(top_k_services)
    average_scores.append(avg_score)

# Plotting
plt.figure(figsize=(10,6))
plt.plot(k_values, average_scores, marker='o', linestyle='-', color='b')
plt.title("Average Rating Score vs. k (#Number of services)")
plt.xlabel("k (#Number of services)")
plt.ylabel("Average Rating Score")
plt.grid(True)
plt.xticks(k_values)
plt.show()


In [ ]:
import pandas as pd

def get_average_score(top_k_services):
    """
    Compute the average rating from the top k services.
    """
    total_score = sum([score for _, score in top_k_services])
    return total_score / len(top_k_services)

# Varying k values
k_values = list(range(2, 11, 2))

# Storing average scores
average_scores = []

for k in k_values:
    top_k_services = cross_network_service_recommendation(final_model, '470500', top_k=k)
    avg_score = get_average_score(top_k_services)
    average_scores.append(avg_score)

# Create DataFrame
df = pd.DataFrame({
    'k': k_values,
    'Average Rating Score': average_scores
})

# Create LaTeX tikzpicture plot from DataFrame
def generate_latex_plot(df, y_column, ylabel):
    coordinates = "\n".join(f"({row['k']}, {row[y_column]})" for _, row in df.iterrows())
    return f"""
\\begin{{tikzpicture}}
\\begin{{axis}}[
    title={{{ylabel} vs. k (#Number of services)}},
    xlabel={{k (#Number of services)}},
    ylabel={{{ylabel}}},
    grid=major,
    legend entries={{{ylabel}}},
]
\\addplot coordinates {{
{coordinates}
}};
\\end{{axis}}
\\end{{tikzpicture}}
"""

# Generating the plot
average_score_plot = generate_latex_plot(df, 'Average Rating Score', 'Average Rating Score')

latex_script = f"""
\\documentclass{{standalone}}
\\usepackage{{pgfplots}}
\\pgfplotsset{{compat=1.15}}

\\begin{{document}}
{average_score_plot}
\\end{{document}}
"""

# Save to a .tex file
with open('average_score_plot.tex', 'w') as file:
    file.write(latex_script)

print("LaTeX script saved to average_score_plot.tex")
